<a href="https://colab.research.google.com/github/vkthakur90/jupyter_notebooks/blob/master/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

In [2]:
!pip install chromadb

  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.10 requires urllib3>=2, but you have urllib3 1.26.18 which is incompatible.


In [3]:
!pip install langchain

In [4]:
!pip install langchainhub

  Using cached urllib3-2.1.0-py3-none-any.whl (104 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.1.0 which is incompatible.


In [5]:
!pip install huggingface_hub

In [6]:
!pip install python-dotenv

In [7]:
import langchain as lc
import langchain.embeddings as lc_embed
import langchain.document_loaders as lcdl
import langchain.text_splitter as lcts
import langchain.vectorstores as lcvs
import langchain.chains as lcchains
import langchain.hub as lchub

In [8]:
import os
import google.colab.files as gcf
import numpy as np
import pandas as pd
import dotenv as denv
import gradio as gr

# **Uploading environment files containing huggingface access token**

In [9]:
!ls

uploaded = gcf.upload()

for k, v in uploaded.items():
    with open(k, 'wb') as f:
        f.write(v)

!ls

flagged  hf.env  sample_data  story.txt


flagged  hf.env  sample_data  story.txt


In [10]:
denv.load_dotenv('hf.env')

True

In [11]:
text_loader = lcdl.TextLoader('story.txt')

# **Mention repository identity of hosted model**

In [12]:
hf_google_flan_t5_xxl = lc.HuggingFaceHub(repo_id = 'google/flan-t5-xxl')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [13]:
sentence_embeddings = lc_embed.HuggingFaceInferenceAPIEmbeddings(
    api_key = os.environ['HUGGINGFACEHUB_API_TOKEN'],
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)

In [14]:
documents = text_loader.load()

In [15]:
text_splitter = lcts.RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 10,
)

In [16]:
splits = text_splitter.split_documents(documents)

In [17]:
db = lcvs.Chroma.from_documents(splits, sentence_embeddings)

In [18]:
retriever = db.as_retriever(search_type="mmr")

# **Create prompt template**

In [19]:
template = """
You are a romantic and erotic novel writer. Write response of question based on following context:

-----------------------------
Context : {context}
-----------------------------

Question: {question}

Answer : The detailed answer to above given question is as follows.
"""

In [20]:
qa_prompt = lc.PromptTemplate.from_template(template)

In [21]:
qa_chain = lcchains.RetrievalQA.from_chain_type(
    llm = hf_google_flan_t5_xxl,
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs={"prompt": qa_prompt}
)

In [22]:
def rag_interface(ques):
    result = qa_chain({
        'query' : ques
    })

    return result['result']

In [23]:
demo = gr.Interface(fn=rag_interface, inputs="text", outputs="text")

In [24]:
demo.launch(share = True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://03ec3322a01c520236.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 561, in process_events
    self.send_message(
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 123, in send_message
    messages = self.pending_messages_per_session[event.session_hash]
KeyError: '5a7elhts6j'


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://03ec3322a01c520236.gradio.live
